In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import datetime
from difflib import SequenceMatcher
import pymysql
from sqlalchemy import create_engine
from tqdm import tqdm
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm

In [16]:
def db_connection(host_name='ds'):
    host_url = "db.ds.mycelebs.com"
    user_nm = "j_eungg"
    passwd = "mycelebs"
    port_num = 3306
    db_name = "star_ko"
    conn = pymysql.connect(host=host_url, user=user_nm, passwd=passwd, port = port_num, charset='utf8',
                           db=db_name, cursorclass=pymysql.cursors.DictCursor)
    # cursorclass=pymysql.cursors.DictCursor 추가 -> 데이터프레임으로 다루기 쉽게 딕셔너리 형태로 결과 반환해줌, cursor는 튜플형태
    # db=db,
    engine = create_engine(f'mysql+pymysql://{user_nm}:{passwd}@{host_url}:{port_num}/{db_name}?charset=utf8mb4')
    return conn, engine

conn, engine = db_connection()

In [17]:
qry = 'SELECT * FROM star_ko_profile_url WHERE naver_url=\'0\''
data = pd.read_sql(qry, conn)

data.head()

In [19]:
len(data)

44

In [20]:
cursor = conn.cursor(pymysql.cursors.DictCursor)

number = int(input("url 수집할 개체 수 입력 :"))
ex = data[0:number]

chrome_options = webdriver.ChromeOptions()
naver_drv = webdriver.Chrome(ChromeDriverManager().install(), options = chrome_options)
profile = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

for i in range(len(ex)-1) :
    profile.execute_script('window.open("about:blank", "_blank");')
    
tabs = profile.window_handles

for i in range(len(ex)) :
    profile.switch_to_window(tabs[i])
    profile.get(f'https://www.mycelebs.ai/celeb/{int(ex["series_id"].iloc[i])}')
    try :
        profile.find_element_by_xpath('//*[@id="closedOneWeek"]').click()
    except :
        pass

for i, row in ex.iterrows() :
    series_id = row['series_id']
    name = row['name']
    
    
    naver_drv.get('https://people.search.naver.com/')
    naver_drv.find_element_by_id('nx_query').send_keys(name)
    naver_drv.find_element_by_class_name('btn_window').click()
    
    print(name, ' - ', series_id)
    naver_url = input("url을 입력하세요(없으면 입력 X): ")
    
    if naver_url == '' :
        qry = f'update star_ko_profile_url set naver_url=\'x\', update_date=NOW() where series_id={series_id};'
        cursor.execute(qry)
        conn.commit()
    else :
        qry = f'update star_ko_profile_url set naver_url=\'{naver_url}\', update_date=NOW() where series_id={series_id};'
        cursor.execute(qry)
        conn.commit()
    
conn.close()
naver_drv.close()
profile.close()

url 수집할 개체 수 입력 :3




====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/jieun/.wdm/drivers/chromedriver/mac64/91.0.4472.101/chromedriver] found in cache


====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/jieun/.wdm/drivers/chromedriver/mac64/91.0.4472.101/chromedriver] found in cache
<ipython-input-20-2f9d149ad2fe>:16: DeprecationWarning: use driver.switch_to.window instead
  profile.switch_to_window(tabs[i])


GI  -  12
url을 입력하세요(없으면 입력 X): https://people.search.naver.com/search.naver?where=nexearch&sm=tab_ppn&query=gi&os=562325&ie=utf8&key=PeopleService
GOT7  -  14
url을 입력하세요(없으면 입력 X): https://people.search.naver.com/search.naver?where=nexearch&sm=tab_ppn&query=got7&os=1706603&ie=utf8&key=PeopleService
강레오  -  29
url을 입력하세요(없으면 입력 X): https://people.search.naver.com/search.naver?where=nexearch&sm=tab_ppn&query=%EA%B0%95%EB%A0%88%EC%98%A4&os=176457&ie=utf8&key=PeopleService
